# Bedrock의 Claude 모델을 활용하여 OCR 텍스트와 원본 이미지를 분석하고, 번역가들이 사용할 최종 파일을 생성하기
(이미지와 OCR 텍스트를 Claude가 분석하여 의미적 그룹으로 분류)

## 1. 환경 셋업

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
!pip install Pillow openpyxl pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [19]:
import sys
import boto3
import json
from typing import Dict, List, Optional
from datetime import datetime
import os
import base64
from botocore.config import Config


# utils 모듈 임포트
from utils import (
    encode_image, 
    read_html_content, 
    get_image_format,
    create_translation_workflow,
    add_python_path,
    check_file_paths,
    list_ocr_results
)

# Python 경로 설정
module_path = ".."
add_python_path(module_path)


python path: /Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery already exists
sys.path:  ['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/joohyery/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages', '/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery']


In [ ]:
# AWS Bedrock 클라이언트 설정

region = "us-west-2"
nova_pro_model_id = "us.amazon.nova-pro-v1:0"
claude3_7_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
claude4_model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"

config = Config(
    read_timeout=300,  # 이미지 + 텍스트 멀티모달 처리를 위해 5분으로 설정
)

client = boto3.client(service_name="bedrock-runtime", region_name=region, config=config)


# 2. OCR 결과 파일 경로 로딩 하기

In [21]:
# 현재 작업 디렉토리 확인
current_folder = os.getcwd()
print(f"현재 작업 디렉토리: {current_folder}")

# 기본 파일 경로
image_path = "samples/6.png"
html_path = "ocr-results-with-upstage/6.html"

# 파일 존재 여부 확인
check_file_paths(image_path, html_path)

현재 작업 디렉토리: /Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-translate
이미지 파일: samples/6.png (존재)
HTML 파일: ocr-results-with-upstage/6.html (존재)


(True, True)

In [ ]:
def llm_converse(image_base64, html_content, model_id):
    """Claude 모델에 Converse API를 사용하여 이미지와 텍스트를 함께 전송하여 분석 요청"""
    
    # 이미지 형식 자동 감지
    image_format = get_image_format(image_path)
    
    # 시스템 프롬프트 (수정된 버전)
    system_prompt = """
    # OCR 텍스트 및 이미지 분석을 통한 레이아웃 기반 텍스트 분류 전문가

    ## 역할 정의
    당신은 OCR 텍스트와 원본 이미지를 분석하여 번역가가 효율적으로 사용할 수 있도록
    텍스트를 레이아웃 기반으로 분류하는 전문가입니다.

    ## 핵심 임무
    1. 정확성 우선: OCR 오류 및 누락 텍스트 수정
    2. 레이아웃 기반 분류: 시각적 배치에 따른 논리적 그룹핑
    3. 번역 효율성 최적화: 번역가의 작업 흐름 개선

    ## 단계별 분석 프로세스

    ### STEP 1: 오류 수정 및 품질 보증
    <thinking>
    이미지와 OCR 텍스트를 비교하여 다음을 수행:
    • 누락된 텍스트 식별 및 추가
    • <td rowspan> 태그가 있는 표 텍스트는 동일 그룹으로 처리
    • 주석이 본문에 섞여있는 경우 분리
    • 문맥상 어색한 부분을 이미지 기준으로 수정
    </thinking>

    수정 기준:
    • 이미지에서 누락된 텍스트 발견 시 → 해당 그룹에 추가
    • <td rowspan> 태그 텍스트 → 표 내 동일 그룹으로 통합
    • 주석이 본문에 포함된 경우 → 주석 제거 후 정리
    • 문맥상 부자연스러운 문장 → 이미지 기준으로 정확한 텍스트 복원

    ### STEP 2: 시각적 레이아웃 분석
    <analysis>
    다음 요소들을 체계적으로 분석:
    1. 공간적 위치 관계 (상하좌우 배치)
    2. 시각적 구분 요소 (폰트 크기, 색상, 스타일)
    3. 구조적 경계선 (박스, 테두리, 구분선)
    4. 계층적 구조 (제목-부제목-본문 관계)
    </analysis>

    ### STEP 3: HTML Figure 요소 특별 처리
    Figure 태그 처리 규칙:
    • <figure> 태그 내용 = 하나의 완전한 논리적 단위
    • <figcaption> + 관련 이미지/차트 = 함께 그룹핑
    • 표, 그래프, 도표의 제목과 데이터 = 연결하여 처리

    ### STEP 4: 그룹핑 전략
    우선순위 기준:
    1. 시각적 근접성 > 의미적 연관성
    2. 동일한 디자인 요소 (배경색, 테두리 등)
    3. 레이아웃상 위치 기반 분류
    4. Figure 요소 별도 그룹 처리

    번역 효율성 고려사항:
    • 번역가의 문맥 이해를 위한 적절한 단위 유지
    • 과도한 세분화 방지
    • 제목/헤더는 독립 그룹으로 처리
    • 의미가 다른 영역은 명확히 분리

    ## 출력 형식

    json
    [
        ["제목 텍스트"],
        ["관련된 본문 텍스트1", "본문 텍스트2"],
        ["figure: 차트 제목", "데이터 항목1", "데이터 항목2", "설명 텍스트"],
        ["다음 섹션 제목"],
        ["해당 섹션 내용1", "내용2"]
    ]


    ## 품질 검증 체크리스트
    • [ ] 모든 OCR 텍스트가 포함되었는가?
    • [ ] 이미지에서 누락된 텍스트가 추가되었는가?
    • [ ] 시각적 레이아웃이 논리적으로 반영되었는가?
    • [ ] Figure 요소가 적절히 그룹화되었는가?
    • [ ] 번역가가 문맥을 이해하기 쉬운 단위로 분류되었는가?
    """
    
    # 사용자 프롬프트 (수정된 버전)
    user_prompt = f"""
    다음은 OCR로 추출된 HTML 내용입니다:
    
    {html_content}
    
    이 HTML에는 figure 태그와 다양한 구조적 요소들이 포함되어 있을 수 있습니다.
    원본 이미지를 주의깊게 관찰하고 다음 사항들을 특별히 고려하여 텍스트를 그룹화해주세요:
    
    1. **시각적 레이아웃**: 텍스트들의 실제 위치와 배치
    2. **Figure 요소**: 표, 차트, 그래프 등과 관련된 텍스트들을 하나의 그룹으로 처리
    3. **계층 구조**: 제목, 부제목, 본문의 시각적 구분
    4. **영역별 구분**: 박스, 배경색, 구분선 등으로 나뉜 영역들
    5. **번역 단위**: 번역가가 작업하기 적절한 크기의 의미 단위
    
    각 그룹은 번역 작업시 하나의 단위로 처리될 것이므로, 문맥상 함께 번역되어야 할 텍스트들을 적절히 묶어주세요.
    """
    
    try:
        # Converse API 호출
        response = client.converse(
            modelId=claude4_model_id,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "image": {
                                "format": image_format,
                                "source": {
                                    "bytes": base64.b64decode(image_base64)
                                }
                            }
                        },
                        {
                            "text": user_prompt
                        }
                    ]
                }
            ],
            system=[
                {
                    "text": system_prompt
                }
            ],
            inferenceConfig={
                "maxTokens": 4000,
                "temperature": 0.1
            }
        )
        
        # 응답에서 텍스트 추출
        response_text = response['output']['message']['content'][0]['text']
        return response_text
        
    except Exception as e:
        raise Exception(f"Claude 모델 호출 중 오류 발생: {str(e)}")

In [ ]:
def main_process_converse(image_path, html_path):
    """전체 프로세스를 실행하는 메인 함수 """
    
    print("=== OCR 텍스트와 이미지 분석 시작 ===")
    
    try:
        # 1. 이미지 인코딩
        print("1. 이미지를 인코딩 중...")
        image_base64 = encode_image(image_path)
        print("   이미지 인코딩 완료")
        
        # 2. HTML 내용 읽기
        print("2. OCR 결과를 읽는 중...")
        html_content = read_html_content(html_path)
        print(f"   HTML 내용 길이: {len(html_content)} 문자")
        
        # 3. 모델로 전사 작업
        print("3. LLM 모델로 텍스트 그룹 분석 중...")
        
        claude_response = llm_converse(image_base64, html_content, claude4_model_id)
        # claude_response = llm_converse(image_base64, html_content, nova_pro_model_id)
        
        print("   Claude 분석 완료")
        
        # 4. JSON 파싱 (수정된 버전)
        print("4. 결과를 파싱 중...")
        
        # Claude 응답이 비어있는지 확인
        if not claude_response or claude_response.strip() == "":
            print("   Claude 응답이 비어있습니다.")
            grouped_texts = [{"category": "General", "texts": ["응답 없음"]}]
        else:
            print(f"   Claude 응답 길이: {len(claude_response)} 문자")
            
            try:
                json_text = claude_response.strip()
                
                # 마크다운 JSON 코드 블록 찾기 (수정된 로직)
                if "```json" in claude_response:
                    json_start = claude_response.find("```json") + 7  # "```json" 다음부터
                    json_end = claude_response.find("```", json_start)
                    
                    if json_end != -1:
                        json_text = claude_response[json_start:json_end].strip()
                    else:
                        json_text = claude_response[json_start:].strip()
                elif "```" in claude_response:
                    json_start = claude_response.find("```") + 3
                    json_end = claude_response.find("```", json_start)
                    
                    if json_end != -1:
                        json_text = claude_response[json_start:json_end].strip()
                    else:
                        json_text = claude_response[json_start:].strip()
                
                print(f"   파싱할 JSON 텍스트: {json_text[:100]}...")
                
                parsed_result = json.loads(json_text)
                
                # 다양한 JSON 구조 처리
                if isinstance(parsed_result, list):
                    grouped_texts = [{"category": f"Group {i+1}", "texts": group} for i, group in enumerate(parsed_result)]
                elif isinstance(parsed_result, dict):
                    if 'groups' in parsed_result:
                        grouped_texts = parsed_result['groups']
                    else:
                        grouped_texts = [{"category": "General", "texts": [str(parsed_result)]}]
                else:
                    grouped_texts = [{"category": "General", "texts": [str(parsed_result)]}]
                
            except json.JSONDecodeError as e:
                print(f"   JSON 파싱 실패: {str(e)}")
                print("   원본 응답을 단순 그룹으로 처리합니다.")
                grouped_texts = [{"category": "General", "texts": [claude_response]}]
        
        print(f"   총 {len(grouped_texts)}개의 텍스트 그룹 생성")
        
        # 5. 번역 문서 생성
        print("5. 번역 문서 생성 중...")
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        final_file_path = create_translation_workflow(
            grouped_texts=grouped_texts,
            image_name=image_name,
            source_lang="Korean",
            target_lang="English"
        )
        
        print("\\n=== 전체 프로세스 완료 ===")
        print(f"최종 파일: {final_file_path}")
        
        return final_file_path, grouped_texts
        
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        return None, None

In [24]:
image_exists, html_exists = check_file_paths(image_path, html_path)

if image_exists and html_exists:
    print("\n" + "="*50)
    print("프로세스를 시작합니다...")
    print("="*50)
    
    print("\n>>> 전사 작업 실행 중...")
    final_file, groups = main_process_converse(image_path, html_path)
    
    if final_file:
        print(f"\n✅ 성공적으로 완료되었습니다!")
        print(f"📁 번역 문서 위치: {final_file}")
        print(f"📊 텍스트 그룹 수: {len(groups) if groups else 0}")
    else:
        print("❌ 프로세스 실행 중 오류가 발생했습니다.")
else:
    print("❌ 필요한 파일이 없어서 프로세스를 실행할 수 없습니다.")

이미지 파일: samples/6.png (존재)
HTML 파일: ocr-results-with-upstage/6.html (존재)

프로세스를 시작합니다...

>>> 전사 작업 실행 중...
=== OCR 텍스트와 이미지 분석 시작 ===
1. 이미지를 인코딩 중...
원본 이미지 크기: 3396x2886
이미지 크기가 적절합니다.
   이미지 인코딩 완료
2. OCR 결과를 읽는 중...
   HTML 내용 길이: 1952 문자
3. LLM 모델로 텍스트 그룹 분석 중...
   Claude 분석 완료
4. 결과를 파싱 중...
   Claude 응답 길이: 663 문자
   파싱할 JSON 텍스트: [
    ["2025.2", "2025.3", "2025.4", "2/6 - 3/4", "슈퍼얼리버드 구매자 한정 혜택", "3/4 - 4/8", "슈퍼얼리버드 구매자 한정 혜택...
   총 9개의 텍스트 그룹 생성
5. 번역 문서 생성 중...
번역 문서를 포맷팅 중...
번역 문서를 저장 중...
번역 문서가 저장되었습니다: /Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-translate/final_results/translation_document_6_20250806_163551.xlsx
총 9개의 텍스트 그룹이 포함되어 있습니다.

=== 번역 문서 생성 완료 ===
파일 경로: /Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-translate/final_results/translation_document_6_20250806_163551.xlsx
텍스트 그룹 수: 9
소스 언어: Korean
타겟 언어: English
\n=== 전체 프로세스 완료 ===
최종 파일: /Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-tra